In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import load_img, img_to_array

In [2]:
path = glob('../ALPR-with-scikit/images/*.xml')
labels_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)


In [3]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv', index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,../ALPR-with-scikit/images\car-wbs-MH01DE2780_...,480,629,206,253
1,../ALPR-with-scikit/images\car-wbs-MH03AR5549_...,1298,4000,712,1612
2,../ALPR-with-scikit/images\car-wbs-MH12FU1014_...,88,192,184,214
3,../ALPR-with-scikit/images\car-wbs-MH43AF5037_...,196,320,177,218
4,../ALPR-with-scikit/images\car-wbs-MH43BU2401_...,585,756,191,239


In [4]:
filename = df['filepath'][0]

def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images/', filename_image)
    return filepath_image

getFilename(filename)

'./images/car-wbs-MH01DE2780_00000.png'

In [5]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]

['./images/car-wbs-MH01DE2780_00000.png',
 './images/car-wbs-MH03AR5549_00000.jpg',
 './images/car-wbs-MH12FU1014_00000.png',
 './images/car-wbs-MH43AF5037_00000.png',
 './images/car-wbs-MH43BU2401_00000.png',
 './images/car-wbs-MH46BV0688_00000.png',
 './images/car-ybs-MH43BP8173_00000.png',
 './images/car-ybs-MH46AD5258_00000.png',
 './images/car-ybs-MH46AD5258_00001.png',
 './images/car-ybs-MH46BF2342_00000.png']

In [6]:
file_path = image_path[0]  # path of our image
img = cv2.imread(file_path)  # read the image
# xmin-480/ymin-206/xmax-629/ymax-253
img = io.imread(file_path)  # Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Image with bounding box')
fig.add_shape(type='rect', x0=480, x1=629, y0=206,y1=253, xref='x', yref='y', line_color='cyan')

In [7]:
labels = df.iloc[:, 1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h, w, d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image, target_size=(224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr / 255.0  # Normalization
    # Normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin / w, xmax / w
    nymin, nymax = ymin / h, ymax / h
    label_norm = (nxmin, nxmax, nymin, nymax)  # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)


In [8]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((523, 224, 224, 3), (131, 224, 224, 3), (523, 4), (131, 4))

In [10]:
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input, outputs=headmodel)

In [11]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [12]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train, y=y_train, batch_size=5, epochs=180, validation_data=(x_test, y_test), callbacks=[tfb])

Epoch 1/180
105/105 [==============================] - 56s 317ms/step - loss: 0.0303 - val_loss: 0.0165
Epoch 2/180
105/105 [==============================] - 27s 255ms/step - loss: 0.0075 - val_loss: 0.0126
Epoch 3/180
105/105 [==============================] - 27s 258ms/step - loss: 0.0045 - val_loss: 0.0180
Epoch 4/180
105/105 [==============================] - 27s 258ms/step - loss: 0.0028 - val_loss: 0.0194
Epoch 5/180
105/105 [==============================] - 27s 258ms/step - loss: 0.0021 - val_loss: 0.0154
Epoch 6/180
105/105 [==============================] - 27s 260ms/step - loss: 0.0015 - val_loss: 0.0166
Epoch 7/180
105/105 [==============================] - 27s 258ms/step - loss: 0.0014 - val_loss: 0.0156
Epoch 8/180
105/105 [==============================] - 27s 256ms/step - loss: 0.0010 - val_loss: 0.0152
Epoch 9/180
105/105 [==============================] - 27s 257ms/step - loss: 8.7161e-04 - val_loss: 0.0156
Epoch 10/180
105/105 [==============================] - 27s 

In [13]:
model.save('./models/object_detection.h5')